In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/30.TecDoc'
else:
    print('fuck')

path_root

'.'

In [2]:
import shutil
directory_out = os.path.join(path_root, './file/6.collate')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

directory_out

'././file/6.collate'

In [3]:
import pandas as pd

from tqdm import tqdm
import json

# 1. Part

In [4]:
directory_in = os.path.join(path_root, './file/2.part')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    df_input = pd.read_excel(os.path.join(directory_in, file),
                             header=0,
                             dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)
    df_input['Packing Unit'] = ['' if packing_unit == '' else int(packing_unit) for packing_unit in df_input['Packing Unit'].tolist()]
    df_input['Quantity Per Packaging Unit'] = ['' if quantity_per_packing_unit == '' else int(quantity_per_packing_unit) for quantity_per_packing_unit in df_input['Quantity Per Packaging Unit'].tolist()]
    df_input['Part Code'] = df_input['Part Code'].astype(int)
    print('Loading Done !')
    print()

    # = = =  = = =  = = =

    list_series, list_column, list_column_information, list_column_criteria = [], df_input.columns.tolist(), [], []
    for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
        dict_dict, dict_dict_2 = json.loads(series['Json_Information']), json.loads(series['Json_Criteria'])
        dict_, dict_2 = {k: v for dict_ in dict_dict.values() for k, v in dict_.items()}, {k: v for dict_ in dict_dict_2.values() for k, v in dict_.items()}

        for key, value in dict_.items():
            if key in list_column:
                key = f'{key} (Information)'

            if key not in list_column_information:
                list_column_information.append(key)

            series[key] = value

        # = = =  = = =  = = =

        for key, value in dict_2.items():
            if key in list_column:
                key = f'{key} (Criteria)'

            if key not in list_column_criteria:
                list_column_criteria.append(key)

            series[key] = value

        list_series.append(series)

    # = = =  = = =  = = =

    print('Data Outputting...')
    df_input = pd.DataFrame(list_series).sort_values(by=['No'],
                                                     ascending=[True],
                                                     ignore_index=True)
    df_input[list_column[:15] + sorted(list_column_information) + sorted(list_column_criteria) + list_column[15:]].to_excel(os.path.join(directory_out, file), index=False)
    print('Output Finished !')
    print()

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !



Progress: 100%|███████████████████████████| 666/666 [00:01<00:00, 420.31it/s]


Data Outputting...
Output Finished !

Data Loading...
Loading Done !



Progress: 100%|████████████████████████| 1381/1381 [00:00<00:00, 1425.27it/s]


Data Outputting...
Output Finished !

Done ~


# 2. Vehicle

In [5]:
directory_in = os.path.join(path_root, './file/5.vehicle')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    df_input = pd.read_excel(os.path.join(directory_in, file),
                             header=0,
                             dtype=str).fillna('')

    df_input['Part Code'] = df_input['Part Code'].astype(int)
    df_input['Make No'] = df_input['Make No'].astype(int)
    df_input['Make Code'] = df_input['Make Code'].astype(int)
    df_input['Link No'] = df_input['Link No'].astype(int)
    print('Loading Done !')
    print()

    # = = =  = = =  = = =

    list_dict, grouped = [], df_input.groupby('Part Code')
    for part_code, df in tqdm(grouped, desc='Progress', ncols=77):
        df = df.reset_index(drop=True)

        # = = =  = = =  = = =

        dict_vehicle = {}
        for i in range(len(df)):
            make, model, engine, year, year_2 = df.loc[i, 'Make'], df.loc[i, 'Model'], '' if df.loc[i, 'cc'] == '' else f'''{df.loc[i, 'cc']}cc''', int(df.loc[i, 'Year'].split('-')[0][:-2]), 2024 if df.loc[i, 'Year'].split('-')[-1] == '' else int(df.loc[i, 'Year'].split('-')[-1][:-2])

            if make not in dict_vehicle:
                dict_vehicle[make] = {model: {engine: [year] if year == year_2 else [year, year_2]}}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = {engine: [year] if year == year_2 else [year, year_2]}
            elif engine not in dict_vehicle[make][model]:
                dict_vehicle[make][model][engine] = [year] if year == year_2 else [year, year_2]
            else:
                if year not in dict_vehicle[make][model][engine]:
                    dict_vehicle[make][model][engine].append(year)
                if year_2 not in dict_vehicle[make][model][engine]:
                    dict_vehicle[make][model][engine].append(year_2)

        # = = =  = = =  = = =

        list_vehicle, list_vehicle_2 = [], []
        for make, dict_model in dict_vehicle.items():
            for model, dict_engine in dict_model.items():
                list_year = [year for list_year in dict_engine.values() for year in list_year]
                year_begin, year_end = min(list_year), max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

                for engine, list_year in dict_engine.items():
                    year_begin, year_end = min(list_year), max(list_year)
                    list_vehicle_2.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

        # = = =  = = =  = = =

        list_dict.append({'Part Code': part_code,
                          'Vehicle W/o Engine': '\n'.join(sorted(list_vehicle)),
                          'Vehicle W/ Engine': '\n'.join(sorted(list_vehicle_2))})

    # = = =  = = =  = = =

    print('Data Outputting...')
    df_output = pd.DataFrame(list_dict).sort_values(by=['Part Code'],
                                                    ascending=[True],
                                                    ignore_index=True)
    df_output.to_excel(os.path.join(directory_out, file), index=False)
    print('Output Finished !')
    print()

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !



Progress: 100%|████████████████████████████| 615/615 [00:11<00:00, 52.47it/s]


Data Outputting...
Output Finished !

Data Loading...
Loading Done !



Progress: 100%|███████████████████████████| 643/643 [00:05<00:00, 117.65it/s]


Data Outputting...
Output Finished !

Done ~
